In [1]:
import pandas as pd
import PyPDF2
import requests
import os
import re
from nltk.corpus import stopwords

In [2]:
df = pd.read_csv('./SearchResults.csv')

print(df.Title.sample(5, random_state=1).values)

df['Query'] = df.Title

stop_words = set(stopwords.words('english'))

df['Query'] = df.Query.apply(lambda x: x.lower())
df['Query'] = df.Query.apply(lambda x: x.replace(',', ' '))
df['Query'] = df.Query.apply(lambda x: x.replace(':', ' '))
df['Query'] = df.Query.apply(lambda x: x.replace('(', ' '))
df['Query'] = df.Query.apply(lambda x: x.replace(')', ' '))
df['Query'] = df.Query.apply(lambda x: x.replace('-', ' '))
df['Query'] = df.Query.apply(lambda x: x.replace('/', ' '))
df['Query'] = df.Query.apply(lambda x: x.replace('?', ' '))
df['Query'] = df.Query.apply(lambda x: x.replace("'", ' '))

df['Query'] = df.Query.apply(lambda x: re.sub('[21][09][0-9]{2}', ' ', x))

df['Query'] = df.Query.apply(lambda x: x.replace('overview', ' '))
df['Query'] = df.Query.apply(lambda x: x.replace('brief', ' '))
df['Query'] = df.Query.apply(lambda x: x.replace('administration', ' '))
df['Query'] = df.Query.apply(lambda x: x.replace('congressional', ' '))
df['Query'] = df.Query.apply(lambda x: x.replace("congress's", ' '))
df['Query'] = df.Query.apply(lambda x: x.replace("congress", ' '))
df['Query'] = df.Query.apply(lambda x: x.replace('introduction', ' '))
df['Query'] = df.Query.apply(lambda x: x.replace('implications', ' '))
df['Query'] = df.Query.apply(lambda x: x.replace('legal', ' '))
df['Query'] = df.Query.apply(lambda x: x.replace('issues', ' '))
df['Query'] = df.Query.apply(lambda x: x.replace('fy', ' '))
df['Query'] = df.Query.apply(lambda x: x.replace('fact', ' '))
df['Query'] = df.Query.apply(lambda x: x.replace('sheet', ' '))
df['Query'] = df.Query.apply(lambda x: x.replace('trends', ' '))
df['Query'] = df.Query.apply(lambda x: x.replace('sketch', ' '))
df['Query'] = df.Query.apply(lambda x: x.replace('primer', ' '))

df['Query'] = df.Query.apply(lambda x: re.sub(r's\.[ ]{1,3}[0-9]{1,5}', ' ', x))
df['Query'] = df.Query.apply(lambda x: re.sub(r'h\.r\.[ ]{1,3}[0-9]{1,5}', ' ', x))

df['Query'] = df.Query.apply(lambda x: re.sub(r'  +', ' ', x))
df['Query'] = df.Query.apply(lambda x: x.strip())

df['Query'] = df.Query.apply(lambda x: ' '.join(filter(lambda x: x not in stop_words, x.split(' '))))
df.Query.sample(5, random_state=1).values

['Effects of Lower Oil Prices'
 "Burma's Political Prisoners and U.S. Sanctions"
 "Veterans' Medical Care Appropriations and Funding Process"
 'Millennium Challenge Corporation: Overview and Issues'
 'Overview of Selected No Surprises Act Litigation']


array(['effects lower oil prices',
       'burma political prisoners u.s. sanctions',
       'veterans medical care appropriations funding process',
       'millennium challenge corporation',
       'selected surprises act litigation'], dtype=object)

In [3]:
df.CoverDate = df.CoverDate.apply(lambda x: pd.to_datetime(x))
df.PublicationDate = df.PublicationDate.apply(lambda x: pd.to_datetime(x))

In [4]:
this_congress = df[df['CoverDate'] >= '2021-01-03']
this_congress = this_congress[this_congress['CoverDate'] <= '2023-01-03']
this_congress = this_congress[this_congress['PublicationDate'] >= '2021-01-03']
this_congress = this_congress[this_congress['PublicationDate'] <= '2023-01-03']

print(len(this_congress))

1759


In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import os
import time

options = Options()
options.headless = True

driver = webdriver.Chrome(options=options)
time.sleep(3)

for i in range(len(this_congress)):
    url = this_congress.Url.iloc[i]

    driver.get(url)

    time.sleep(3)

driver.quit()

# H.Con.Res.

Should also grab surrounding text for further analysis

Add empty dot capture groups before and after the main group


In [6]:
crs = os.listdir('./crs_reports/')

In [7]:
bills = []

In [8]:
for crs_index in range(len(crs)):
    print(crs_index, crs[crs_index])
# for crs_index in range(30):
    senate_bills = []
    house_bills = []
    h_con_res = []
    with open('./crs_reports/' + crs[crs_index], 'rb') as file:
        pdf = PyPDF2.PdfReader(file)
        
        for page_index in range(len(pdf.pages)):
            pageObj = pdf.pages[page_index]
            
            text = pageObj.extract_text()

            for item in re.finditer(r'H\.R\.[ ]{0,5}[0-9]+', text):
                start, end = item.span()
                start = start - 200  # Expand match window to get surrounding context
                end = end + 200
                house_bills.append((item.group(), text[start:end]))

            for item in re.finditer(r'\sS\.[ ]{0,5}[0-9]+', text):
                start, end = item.span()
                start = start - 200  # Expand match window to get surrounding context
                end = end + 200
                senate_bills.append((item.group(), text[start:end]))

            for item in re.finditer(r'H\.Con\.Res\.[ ]{0,5}[0-9]+', text):
                start, end = item.span()
                start = start - 200  # Expand match window to get surrounding context
                end = end + 200
                h_con_res.append((item.group(), text[start:end]))

    cover_date, publication_date, title, query = df[df.ProductNumber == crs[crs_index].split('.')[0]][['CoverDate', 'PublicationDate', 'Title', 'Query']].values.tolist()[0]

    for bill in house_bills:
        bills.append({
            'crs_report_file_name': crs[crs_index],
            'crs_report_index': crs_index,
            'bill_type': 'hr',
            'bill_title': bill[0],
            'surrounding_text': bill[1],
            'cover_date': cover_date,
            'publication_date': publication_date,
            'title': title,
            'query': query
        })

    for bill in senate_bills:
        bills.append({
            'crs_report_file_name': crs[crs_index],
            'crs_report_index': crs_index,
            'bill_type': 's',
            'bill_title': bill[0],
            'surrounding_text': bill[1],
            'cover_date': cover_date,
            'publication_date': publication_date,
            'title': title,
            'query': query
        })

    for bill in h_con_res:
        bills.append({
            'crs_report_file_name': crs[crs_index],
            'crs_report_index': crs_index,
            'bill_type': 'hconres',
            'bill_title': bill[0],
            'surrounding_text': bill[1],
            'cover_date': cover_date,
            'publication_date': publication_date,
            'title': title,
            'query': query
        })


0 IF11712.pdf
1 IF11713.pdf
2 IF11718.pdf
3 IF11720.pdf
4 IF11721.pdf
5 IF11722.pdf
6 IF11723.pdf
7 IF11724.pdf
8 IF11725.pdf
9 IF11726.pdf
10 IF11730.pdf
11 IF11731.pdf
12 IF11732.pdf
13 IF11733.pdf
14 IF11734.pdf
15 IF11736.pdf
16 IF11738.pdf
17 IF11739.pdf
18 IF11740.pdf
19 IF11742.pdf
20 IF11743.pdf
21 IF11746.pdf
22 IF11747.pdf
23 IF11748.pdf
24 IF11750.pdf
25 IF11752.pdf
26 IF11753.pdf
27 IF11756.pdf
28 IF11759.pdf
29 IF11761.pdf
30 IF11762.pdf
31 IF11763.pdf
32 IF11764.pdf
33 IF11765.pdf
34 IF11766.pdf
35 IF11767.pdf
36 IF11768.pdf
37 IF11769.pdf
38 IF11770.pdf
39 IF11771.pdf
40 IF11774.pdf
41 IF11775.pdf
42 IF11776.pdf
43 IF11777.pdf
44 IF11778.pdf
45 IF11779.pdf
46 IF11781.pdf
47 IF11782.pdf
48 IF11784.pdf
49 IF11785.pdf
50 IF11786.pdf
51 IF11787.pdf
52 IF11789.pdf
53 IF11790.pdf
54 IF11791.pdf
55 IF11792.pdf
56 IF11793.pdf
57 IF11794.pdf
58 IF11795.pdf
59 IF11796.pdf
60 IF11798.pdf
61 IF11799.pdf
62 IF11800.pdf
63 IF11801.pdf
64 IF11802.pdf
65 IF11803.pdf
66 IF11805.pdf
67 IF

In [10]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import statistics

bert_model = AutoModelForSequenceClassification.from_pretrained(
    'C:/Users/jackv/Documents/GitHub/search-engines/hw1/QryEval/INPUT_DIR/ms-marco-MiniLM-L-12-v2', 
    num_labels=1
)
bert_model.eval()
bert_tokenizer = AutoTokenizer.from_pretrained('C:/Users/jackv/Documents/GitHub/search-engines/hw1/QryEval/INPUT_DIR/ms-marco-MiniLM-L-12-v2')

def bert_score(query, passage):
    bert_input =  bert_tokenizer.encode_plus(
            [query, passage],		
            add_special_tokens=True,
            max_length=512,
            truncation="only_second",	
            return_tensors="pt")

    with torch.no_grad():
        outputs = bert_model(**bert_input) 
        score = outputs.logits.data.item()
        return(score)	

In [14]:
import sqlite3
import numpy as np

pd.set_option('display.max_columns', None)

conn = sqlite3.Connection('./congress-data_v2.0.db')

s_pattern = r'S.*:(.*)'
hr_pattern = r'HR.*:(.*)'
hconres_pattern = r'HConRes.*:(.*)'

matches = []

for i in range(len(bills)):
    print(i, bills[i]['bill_title'])
    bill_code = bills[i]['bill_title'].strip()
    crs_title = bills[i]['title']  # change to query
    context = bills[i]['surrounding_text']
    context = context.split(' ')
    context = context[len(context) // 4: 3 * len(context) // 4]
    context = ' '.join(context)
    context = crs_title + ' ' + context

    if bill_code.startswith('S'):
        bill_number = re.match(r'S\.[ ]{0,5}([0-9]+)', bill_code).groups()[0]
        results = pd.read_sql(f"select id, title from full_texts where file_number = {bill_number} and file_chamber = 's' and title != 'None'", conn)
        clean_titles = results.title.apply(lambda x: re.findall(s_pattern, x)[0]).values.tolist()
        ids = results['id'].values.tolist()
        bert_scores = []
        if clean_titles:
            for title in clean_titles:
                bert_scores.append(bert_score(context, title))
            matches.append({
                'best_bert_score': np.max(bert_scores),
                'full_text_match_pk': ids[np.argmax(bert_scores)],
                'bill_code': bill_code,
                'crs_title': crs_title,
                'context': context,
                'matching_title': clean_titles[np.argmax(bert_scores)],
                'options': list(zip(clean_titles, bert_scores))
            })
        
    elif bill_code.startswith('H.R'):
        bill_number = re.match(r'H\.R\.[ ]{0,5}([0-9]+)', bill_code).groups()[0]
        results = pd.read_sql(f"select id, title from full_texts where file_number = {bill_number} and file_chamber = 'hr' and title != 'None'", conn)
        clean_titles = results.title.apply(lambda x: re.findall(hr_pattern, x)[0]).values.tolist()
        ids = results['id'].values.tolist()
        bert_scores = []
        if clean_titles:
            for title in clean_titles:
                bert_scores.append(bert_score(context, title))
            matches.append({
                'best_bert_score': np.max(bert_scores),
                'full_text_match_pk': ids[np.argmax(bert_scores)],
                'bill_code': bill_code,
                'crs_title': crs_title,
                'context': context,
                'matching_title': clean_titles[np.argmax(bert_scores)],
                'options': list(zip(clean_titles, bert_scores))
            })

    elif bill_code.startswith('H.Con.Res'):
        continue
        # if not re.match(r'H\.Con\.Res\.[ ]{0,5}([0-9]+)', bill_code):
        #     print('No match for', bill_code)
        #     continue
        # bill_number = re.match(r'H\.Con\.Res\.[ ]{0,5}([0-9]+)', bill_code).groups()[0]
        # results = pd.read_sql(f"select id, title from full_texts where file_number = {bill_number} and file_chamber = 'hconres' and title != 'None'", conn)
        # clean_titles = results.title.apply(lambda x: re.findall(hconres_pattern, x)[0]).values.tolist()
        # ids = results['id'].values.tolist()
        # bert_scores = []
        # if clean_titles:
        #     for title in clean_titles:
        #         bert_scores.append(bert_score(context, title))
        #     matches.append({
        #         'best_bert_score': np.max(bert_scores),
        #         'full_text_match_pk': ids[np.argmax(bert_scores)],
        #         'bill_code': bill_code,
        #         'crs_title': crs_title,
        #         'context': context,
        #         'matching_title': clean_titles[np.argmax(bert_scores)],
        #         'options': list(zip(clean_titles, bert_scores))
        #     })

    else:
        print('ValueError:', bill_code)
        raise ValueError ('Unexpected bill title value.')

0 H.R. 419
1 H.R. 374
2 H.R. 7608
3 H.R. 7612
4 H.R. 7608
5 H.R. 7608
6 H.R. 1319
7 H.R. 7609
8 H.R. 7608
9 H.R. 7608
10 H.R. 8
11 H.R. 1446
12 H.R. 8
13 H.R. 1446
14 H.R. 8
15 H.R. 1112
16 H.R. 8
17 H.R. 1112
18 H.R. 8
19 H.R. 1446
20 H.R. 8
21 H.R. 8
22 H.R. 8
23 H.R. 8
24 H.R. 8
25 H.R. 1446
26 H.R. 1446
27 H.R. 1446
28 H.R. 1446
29 H.R. 1112
30 H.R. 1112
31 H.R. 8
32 H.R. 1446
33  S. 529
34 H.R. 1620
35 H.R. 1620
36 H.R. 1620
37 H.R. 1620
38 H.R. 1620
39 H.R. 1585
40 H.R. 1620
41 H.R. 1620
42 H.R. 1620
43 H.R. 1620
44 H.R. 1620
45 H.R. 1620
46 H.R. 1620
47 H.R. 1620
48  S. 3623
49  S. 3623
50  S. 3623
51 H.R. 1522
52 H.R. 2070
53 H.R. 8393
54  S. 780
55  S. 865
56 H.R. 1848
57 H.R. 906
58  S. 82
59 H.R. 1
60 H.R. 1
61 H.R. 5376
62 H.R. 1785
63 H.R. 1785
64 H.R. 1785
65 H.R. 1785
66 H.R. 2976
67 H.R. 1786
68 H.R. 1786
69 H.R. 1786
70  S. 20
71  S. 714
72  S. 714
73  S. 991
74  S. 991
75 
S. 714
76  S. 991
77 
S. 714
78  S. 991
79  S. 991
80  S. 991
81  S. 714
82  S. 1501
83  S. 991


In [15]:
pd.DataFrame(matches).to_csv('crs_matches_11-15-23.csv', index=False)

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity
# from sklearn.cluster import DBSCAN
# import numpy as np

# # Example list of strings
# strings = this_congress.Query.values.tolist()

# # Step 1 & 2: Define similarity metric and calculate similarity matrix
# vectorizer = TfidfVectorizer().fit_transform(strings)
# similarity_matrix = cosine_similarity(vectorizer)

# # Convert similarity to distance
# distance_matrix = 1 - similarity_matrix
# distance_matrix[distance_matrix < 0] = 0

# # Step 3: Cluster the strings
# db = DBSCAN(eps=0.5, min_samples=2, metric="precomputed")
# clusters = db.fit_predict(distance_matrix)

# # Step 4: Identify the largest clusters
# cluster_sizes = np.bincount(clusters + 1)  # +1 to handle noise label (-1)

# # Sort cluster sizes (excluding noise) and get indexes
# sorted_cluster_indexes = np.argsort(cluster_sizes[1:])[::-1]  # Reverse for descending order

# # Get the second largest cluster index
# if len(sorted_cluster_indexes) > 1:
#     second_largest_cluster_index = sorted_cluster_indexes[2]
#     second_largest_cluster = np.where(clusters == second_largest_cluster_index)[0]
#     for i in second_largest_cluster:
#         print(strings[i])
# else:
#     print("There is no second largest cluster.")

electricity transmission
selected health provisions inflation reduction act
u.s. environmental protection agency epa appropriations president budget request
tax provisions inflation reduction act
judiciary appropriations
u.s. environmental protection agency epa appropriations
u.s. environmental protection agency epa water infrastructure programs appropriations
child welfare president budget
judiciary budget request
child welfare programs timeline
judiciary appropriations
u.s. environmental protection agency epa water infrastructure programs appropriations
irs related funding inflation reduction act
offshore wind provisions inflation reduction act
inflation reduction act incentives clean transportation
distribution irs enforcement activity funded inflation reduction act
clean vehicle tax credits inflation reduction act
electricity transmission provisions inflation reduction act
inflation reduction act u.s. environmental protection agency selected environmental provisions
ndaa president 

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity
# import numpy as np

# def find_top_similar_strings_cosine(str_list, top_n=20):
#     vectorizer = TfidfVectorizer()
#     tfidf_matrix = vectorizer.fit_transform(str_list)
#     cosine_sim = cosine_similarity(tfidf_matrix)

#     np.fill_diagonal(cosine_sim, 0)
#     scores_and_indices = sorted(
#         ((score, i, j) for i, row in enumerate(cosine_sim) 
#                       for j, score in enumerate(row) if i != j),
#         reverse=True,
#         key=lambda x: x[0]
#     )[:top_n]

#     top_pairs = [(str_list[i], str_list[j], score) for score, i, j in scores_and_indices]

#     return top_pairs



# top_pairs = find_top_similar_strings_cosine(df.Query.values.tolist(), top_n=20)

# for i, (str1, str2, score) in enumerate(top_pairs, 1):
#     print(f"{i}. '{str1}' and '{str2}' with a cosine similarity of {score}")



GPT-based query creation (discarded in favor of title-based queries)

In [ ]:
# import openai
# openai.api_key = os.environ.get('OPENAI_API_KEY')

# text = ''
# with open('./crs_reports/' + crs[646], 'rb') as file:
#     pdf = PyPDF2.PdfReader(file)
#     for page_index in range(len(pdf.pages)):
#         pageObj = pdf.pages[page_index]
#         text = text + '\n' + pageObj.extract_text()

# text = text.split(' ')
# text = text[:1000]
# text = ' '.join(text)

# chat = [ 
#     {
#         'role': 'system', 
#         'content': f"Document:\n\"{text}\"\nWrite a very short search query to describe this document." 
#     }
# ]

# completion = openai.ChatCompletion.create(
#     model='gpt-3.5-turbo-1106',
#     messages=chat
# )

# # Remove "Congressional Research Service", "Overview"

Keyword based query generation (discarded in favor of title-based)

In [ ]:
# from collections import Counter 
# from nltk.corpus import stopwords
# from nltk.stem import PorterStemmer
# from nltk.tokenize import word_tokenize
# from nltk.stem import WordNetLemmatizer
# from nltk.tokenize import word_tokenize
# import nltk

# text = ''
# with open('./crs_reports/' + crs[646], 'rb') as file:
#     pdf = PyPDF2.PdfReader(file)
#     for page_index in range(len(pdf.pages)):
#         pageObj = pdf.pages[page_index]
#         text = text + '\n' + pageObj.extract_text()

# nltk.download('wordnet')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

# lemmatizer = WordNetLemmatizer()

# strings = text.split(' ')

# lemmatized_strings = []
# for string in strings:
#     tokens = word_tokenize(string)
#     lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
#     lemmatized_strings = lemmatized_strings + lemmatized_tokens

# lemmatized_strings = [x.lower() for x in lemmatized_strings]

# counter = Counter(lemmatized_strings)

# stop_words = set(stopwords.words('english'))

# for stop_word in stop_words:
#     counter.pop(stop_word, None)

# counter.pop('', None)
# counter.pop(',', None)
# counter.pop('.', None)
# counter.pop(')', None)
# counter.pop('(', None)
# counter.pop('’', None)
# counter.pop('$', None)
# counter.pop('“', None)
# counter.pop('”', None)
# counter.pop('•', None)

# counter.pop('u.s.')
# counter.pop('u.s')

# ' '.join([x[0].strip() for x in sorted(dict(counter).items(), key=lambda x: x[1], reverse=True)[:5]])

[nltk_data] Downloading package wordnet to C:\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to C:\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


'contract myp procurement multiyear defense'